# Logistic Regression for E-Commerce

In the [AB_Test notebook](http://localhost:8888/notebooks/notebooks/AB_Test.ipynb), we analysed the results of an A/B test experiment run by an e-commerce website. We could not reject the null hypothesis that states, "The conversion rate for the new webpage is the same or worse than the conversion rate for the old page" based on a 5% type 1 error rate and made the recommendation that the new page is not implemented. 

In this notebook, we build a logisitic regression model to predict the probability of individual user conversion. This model may help the e-commerce site understand which users are most and least likely to make a purchase and implement their marketing strategies accordingly.

## Introduction
An e-commerce company has developed a new web page in order to try and increase purchases (conversions) made through their site. They have run an AB experiment and collected data regarding the number of conversions for a control group and a treatment group. The objective of this project is to assess the data and help the company understand if they should implement this new page, keep the old page, or run the experiment longer to make their decision.



## Data Overview
The datasets for this project are provided by **Udacity** in association with their **Data Analyst Nanodegree** programme. They have been generated to represent example A/B experiment results from an e-commerce website. There are 2 raw csv files:

  * `ab_test.csv` - contains information regarding the group (control or treatment), landing page (old or new), timestamp of visit and conversion.
  * `countries.csv` - contains information regarding the country of origin of each user.
  
  ***
## Regression Analysis
Given that user conversion is binary, another method we can use to investigate the probability of conversion is logistic regression. To reinforce conceptual understanding of logistic regression algorithms we will build them from scratch then compare and optimise performance. 

Model plan:
* Engineer features in ab_test_clean.csv
* Divide the dataset randomly into a 70:30 training and test set split
* Train unregularised logistic regression model. Assess performance against mutliple learning rates $\alpha$
* Train regularised logistic regression model. Compare training accuracy for multiple values of $\lambda$